In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import pandas as pd
import os
import plotly.graph_objects as go

# Definir o caminho para o diretório onde os arquivos Excel estão localizados
path = "/content/drive/MyDrive/preprocessed/"  # Atualize o caminho conforme necessário

# Função para ler as planilhas
def ler_dados(file):
    try:
        dados = pd.read_excel(file)
        dados = dados.iloc[:, 1:]  # Seleciona todas as colunas, exceto a primeira
        numeric_cols = dados.select_dtypes(include='number')

        # Padronizando os dados
        for col in numeric_cols.columns:
            media_col = numeric_cols[col].mean()
            desvio_padrao_col = numeric_cols[col].std()
            numeric_cols[col] = (numeric_cols[col] - media_col) / desvio_padrao_col

        # Calcular a matriz de correlação
        correlacao = numeric_cols.corr()
        return numeric_cols, correlacao
    except Exception as e:
        print(f"Erro ao ler o arquivo {file}: {e}")
        return pd.DataFrame(), pd.DataFrame()

# Função para obter as informações do poço
def obter_informacoes_poco(arquivo):
    info_pocos = pd.DataFrame({
        "Poço": ["3-BRSA-1148-SE", "3-BRSA-1098-SE", "3-BRSA-1093-SE", "3-SZ-464-SE", "3-BRSA-912-SE",
                 "3-BRSA-1279-SE", "3-BRSA-1262-SE", "3-BRSA-1307D-SE", "6-BRSA-1202D-BA", "1-BRSA-470D-BA",
                 "4-BRSA-422-BA", "3-BRSA-349-RN", "3-BRSA-1295-RN", "3-BRSA-1193-RN", "3-BRSA-1224-RN",
                 "3-BRSA-1210-RN", "3-BRSA-1325-RN", "3-BRSA-1320-RN", "3-BRSA-1341-RN", "3-OGX-95-MA",
                 "4-PGN-25-MA", "1-OGX-117-MA", "4-PGN-19-MA", "7-GVB-8-MA", "7-GVB-6-MA", "7-RUC-59DP-AM",
                 "3-BRSA-515-AM", "3-BRSA-1057D-AM", "3-BRSA-670-AM", "3-BRSA-1121-AM","3-BRSA-1057D"],
        "Campo": ["Siririzinho", "Siririzinho", "Siririzinho", "Siririzinho", "Carmópolis",
                  "Carmópolis", "Carmópolis", "Carmópolis", "Dom João", "Guanambi",
                  "CEXIS", "Canto do Amaro", "Canto do Amaro", "Canto do Amaro", "Estreito",
                  "Estreito", "Estreito", "Alto do Rodrigues", "Alto do Rodrigues", "Gavião Branco",
                  "Gavião Tesoura", "Gavião Tesoura", "Gavião Tesoura", "Gavião Branco", "Gavião Branco",
                  "Rio Urucu", "Rio Urucu", "Rio Urucu", "Rio Urucu", "Rio Urucu","NA"],
        "Bacia": ["Sergipe", "Sergipe", "Sergipe", "Sergipe", "Sergipe",
                  "Sergipe", "Sergipe", "Sergipe", "Recôncavo", "Recôncavo",
                  "Recôncavo", "Potiguar", "Potiguar", "Potiguar", "Potiguar",
                  "Potiguar", "Potiguar", "Potiguar", "Potiguar", "Parnaíba",
                  "Parnaíba", "Parnaíba", "Parnaíba", "Parnaíba", "Parnaíba",
                  "Solimões", "Solimões", "Solimões", "Solimões", "Solimões","NA"]
    })

    info = info_pocos[info_pocos["Poço"] == arquivo.replace(".xlsx", "")]
    return info

# Inicializando a aplicação Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout da aplicação
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Análise de Perfilagem dos 30 Poços de Petróleo", className="text-center text-primary mb-4"), width=12)
    ]),
    dbc.Row([
        dbc.Col([
            html.Label("Selecione o poço:"),
            dcc.Dropdown(
                id='arquivo-dropdown',
                options=[{'label': f, 'value': f} for f in os.listdir(path) if f.endswith('.xlsx')],
                value=None
            ),
            dbc.Button("Atualizar", id="atualizar-btn", color="success", className="mt-3")
        ], width=3),
        dbc.Col([
            html.Div(id='campo-info', className='mt-3'),
            html.Div(id='bacia-info', className='mt-2'),
        ], width=9)
    ]),
    dbc.Row([
        dbc.Col([dcc.Graph(id='plot-linhas')], width=6),
        dbc.Col([dcc.Graph(id='plot-correlacao')], width=6)
    ]),
    dbc.Row([
        dbc.Col([
            html.H4("Estatísticas Descritivas"),
            html.Pre(id='estatisticas')
        ], width=12)
    ])
])

# Callback para atualizar o conteúdo ao clicar em "Atualizar"
@app.callback(
    [Output('plot-linhas', 'figure'),
     Output('plot-correlacao', 'figure'),
     Output('estatisticas', 'children'),
     Output('campo-info', 'children'),
     Output('bacia-info', 'children')],
    [Input('atualizar-btn', 'n_clicks')],
    [State('arquivo-dropdown', 'value')]
)
def atualizar_analise(n_clicks, arquivo):
    if not arquivo:
        return {}, {}, "Selecione um poço", "", ""

    # Carregar dados e calcular a correlação
    file_path = os.path.join(path, arquivo)
    dados_padronizados, correlacao = ler_dados(file_path)

    # Verifique se há uma coluna de profundidade no conjunto de dados e use-a como eixo X
    if 'PROF (m)' in dados_padronizados.columns:
        eixo_x = dados_padronizados['PROF (m)']
    else:
        eixo_x = dados_padronizados.index  # Caso não haja profundidade, usar o índice como fallback

    # Gráfico de Linhas (Usando profundidade como eixo X)
    fig_linhas = go.Figure()
    for coluna in dados_padronizados.columns:
        if coluna != 'PROF (m)':  # Não incluir a profundidade como uma variável a ser plotada
            fig_linhas.add_trace(go.Scatter(x=eixo_x, y=dados_padronizados[coluna], mode='lines', name=coluna, line=dict(width=2)))

    fig_linhas.update_layout(
        title=f"Visualização das Variáveis Padronizadas ao Longo da Profundidade - {arquivo}",
        xaxis_title="Profundidade (m)",
        yaxis_title="Valores Padronizados",
        template="plotly_white",
        legend_title="Variáveis",
        hovermode="x unified"
    )

    # Matriz de Correlação (Adicionando rótulos e melhorando as cores)
    fig_corr = go.Figure(data=go.Heatmap(
        z=correlacao.values,
        x=correlacao.columns,
        y=correlacao.index,
        colorscale='RdBu',
        zmid=0,
        text=correlacao.values,
        hoverinfo='text'
    ))

    fig_corr.update_layout(
        title="Matriz de Correlação",
        template="plotly_white"
    )

    # Estatísticas Descritivas
    estatisticas = dados_padronizados.describe().round(4).to_string()

    # Obter informações do poço
    info_poco = obter_informacoes_poco(arquivo)
    campo_info = f"Campo de Produção: {info_poco['Campo'].values[0]}"
    bacia_info = f"Bacia Sedimentar: {info_poco['Bacia'].values[0]}"

    return fig_linhas, fig_corr, estatisticas, campo_info, bacia_info

# Rodar a aplicação usando o Proxy do Colab
if __name__ == '__main__':
    app.run_server(port=8050, debug=True)

    # Gerar o link de acesso ao dashboard
    from google.colab.output import eval_js
    print(eval_js("google.colab.kernel.proxyPort(8050)"))

<IPython.core.display.Javascript object>

https://2pr28njcdsc-496ff2e9c6d22116-8050-colab.googleusercontent.com/
